Lunar Lander sin LR y sin GAEs ni buffer

In [ ]:
import os

import random
import gym
import pylab
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

#tf.config.experimental_run_functions_eagerly(True)
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import backend as K
import copy

EPISODES = 100000 # Number of times the enviroment is ran
LR = 0.00025 # Learning rate 
NUM_ACTIONS = 4 # Number of possible action in the environment
NUM_STATES = 8 # Number of possible states in the exvironment
EPOCHS = 10 # Epochs to train the network (recommended between 3 and 30)
BATCH_SIZE = 64 # Batch size for the neural nets
BUFFER_SIZE = 2048 # Buffer of experiences
SHUFFLE = True # Whether to shuffle data or not while training
OPTIMIZER = Adam # Optimizer for both actor and critic
GAMMA = 0.99 # Used for the estimated reward
NORMALIZE = True # Whether to normalize GAE or not


# Create the actor used to select the action given an state
class Actor_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Softmax as there are different probabilities depending on the action
        output = Dense(NUM_ACTIONS, activation="softmax")(X)
        
        # Compile the model with the custom loss
        self.model = Model(inputs = X_input, outputs = output)
        self.model.compile(loss='categorical_crossentropy', optimizer=OPTIMIZER(lr=LR))

# Create the critic which will criticise how the actor is performing    
class Critic_Model:
    def __init__(self):
        X_input = Input(NUM_STATES)

        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X_input)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        X = Dense(64, activation="relu", kernel_initializer='he_uniform')(X)
        
        # Linear output to know how good the action is
        value = Dense(1)(X)
        
        # Compile it with mse loss and gradient descent
        self.model = Model(inputs=X_input, outputs = value)
        self.model.compile(loss='mse', optimizer=OPTIMIZER(lr=LR))

# Combine both Actor and Critic to create the agent
class PPOAgent:
    def __init__(self, env_name):
        # Environment parameters
        self.env_name = env_name       
        self.env = gym.make(env_name)
        self.episode = 0 # used to track current number episoded since start
        self.max_average = 0 # record max average reached
        
        # Used to plot a grapgh of the train process
        self.scores_, self.average_ = [], []

        # Create Actor-Critic network models
        self.Actor = Actor_Model()
        self.Critic = Critic_Model()
    
        # Names for the models
        self.Actor_name = f"{self.env_name}_PPO_Actor.h5"
        self.Critic_name = f"{self.env_name}_PPO_Critic.h5"

    # Get the action given the current state    
    def act(self, state):
        # Use the network to predict the next action to take, using the model
        prediction = self.Actor.model.predict(state)[0]
        
        # Probability based to choose the action
        action = np.random.choice(NUM_ACTIONS, p=prediction)
        action_onehot = np.zeros([NUM_ACTIONS])
        action_onehot[action] = 1
        return action, action_onehot, prediction
    
    def discount_rewards(self, reward):#gaes is better
        # Compute the gamma-discounted rewards over an episode
        # We apply the discount and normalize it to avoid big variability of rewards
        running_add = 0
        discounted_r = np.zeros_like(reward)
        for i in reversed(range(0,len(reward))):
            running_add = running_add * GAMMA + reward[i]
            discounted_r[i] = running_add

        discounted_r -= np.mean(discounted_r) # normalizing the result
        discounted_r /= (np.std(discounted_r) + 1e-8) # divide by standard deviation
        return discounted_r
    
    def replay(self, states, actions, rewards, predictions, dones, next_states):
        # Reshape memory to appropriate shape for training
        states = np.vstack(states)
        next_states = np.vstack(next_states)
        actions = np.vstack(actions)
        predictions = np.vstack(predictions)

        # Get Critic network predictions for state and next state
        values = self.Critic.model.predict(states)
        next_values = self.Critic.model.predict(next_states)

        # Compute de discounted reward
        discounted_rewards = self.discount_rewards(rewards)

        # Stack info to unpack it in the custom loss
        y_true = actions

        values = np.reshape(values, (np.shape(values)[0],))

        advantages = np.vstack(discounted_rewards - values)
        advantages = np.reshape(advantages, (np.shape(advantages)[0],))

        start_train = time.time()
        # Training Actor and Critic networks
        a_loss = self.Actor.model.fit(states, y_true, sample_weight=advantages, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        c_loss = self.Critic.model.fit(states, discounted_rewards, epochs=EPOCHS, verbose=0, shuffle=SHUFFLE, batch_size=BATCH_SIZE)
        #print('Time: ', time.time()-start_train)
 
    def load(self):
        self.Actor.Actor.load_weights(self.Actor_name)
        self.Critic.Critic.load_weights(self.Critic_name)

    def save(self):
        self.Actor.model.save_weights(self.Actor_name)
        self.Critic.model.save_weights(self.Critic_name)

    def run_batch(self): # train every self.Training_batch episodes
        global LR
        state = self.env.reset()
        state = np.reshape(state, [1, NUM_STATES])
        done, score = False, 0
        finished = False
        while finished == False:
            # Instantiate or reset games memory
            states, next_states, actions, rewards, predictions, dones = [], [], [], [], [], []
            for t in range(BUFFER_SIZE):
                #self.env.render()
                # Actor picks an action
                action, action_onehot, prediction = self.act(state)
                # Retrieve new state, reward, and whether the state is terminal
                next_state, reward, done, _ = self.env.step(action)
                # Memorize (state, action, reward) for training
                states.append(state)
                next_states.append(np.reshape(next_state, [1, NUM_STATES]))
                actions.append(action_onehot)
                rewards.append(reward)
                dones.append(done)
                predictions.append(prediction)
                # Update current state
                state = np.reshape(next_state, [1, NUM_STATES])
                score += reward
                if done:
                    self.episode += 1
                    self.scores_.append(score)
                    #average, SAVING = self.PlotModel(score, self.episode)
                    #print("episode: {}/{}, score: {}, average: {:.2f} {}".format(self.episode, EPISODES, score, average, SAVING))
                    if self.episode >= 100:
                        average = sum(self.scores_[-100:])/100
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, average))
                        if average > self.max_average:
                            self.max_average = average
                            if self.max_average > 150:
                                self.save()
                            LR *= 0.95
                            K.set_value(self.Actor.model.optimizer.learning_rate, LR)
                            K.set_value(self.Critic.model.optimizer.learning_rate, LR)
                            
                        if average > 200:
                            plt.plot(self.scores_)
                            plt.xlabel("Episode")
                            plt.ylabel("Score")
                            finished = True
                            break

                    else:
                        print('Episode: {:>5}\t\tscore: {:>7.2f}\t\taverage: {:>7.2f}'.format(self.episode, score, sum(self.scores_)/len(self.scores_)))
                    
                    
                    state, done, score = self.env.reset(), False, 0
                    state = np.reshape(state, [1, NUM_STATES])
                    
            self.replay(states, actions, rewards, predictions, dones, next_states)
            if self.episode >= EPISODES:
                break
        self.env.close()   
                    
if __name__ == "__main__":
    start = time.time()
    env_name = 'LunarLander-v2'
    agent = PPOAgent(env_name)
    #agent.run() # train as PPO, train every epesode
    agent.run_batch() # train as PPO, train every batch, trains better
    #agent.run_multiprocesses(num_worker = 8)  # train PPO multiprocessed (fastest)
    #agent.test()
    print((time.time() - start)/60)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Episode:     1		score: -515.22		average: -515.22
Episode:     2		score:  -66.83		average: -291.03
Episode:     3		score: -420.96		average: -334.34
Episode:     4		score: -425.32		average: -357.08
Episode:     5		score: -531.17		average: -391.90
Episode:     6		score: -223.79		average: -363.88
Episode:     7		score: -243.15		average: -346.63
Episode:     8		score: -483.32		average: -363.72
Episode:     9		score:  -81.87		average: -332.40
Episode:    10		score: -427.84		average: -341.95
Episode:    11		score: -396.73		average: -346.93
Episode:    12		score: -301.41		average: -343.14
Episode:    13		score: -384.82		average: -346.34
Episode:    14		score: -518.17		average: -358.62
Episode:    15		score: -440.03		average: -364.04
Episode:    16		score: -415.43		average: -367.25
Episode:    17		score: -358.86		average: -366.76
Episode:    18		score: -402.19		average: -368.73
Episode:    19		score: -319.14		avera

Episode:   162		score: -576.10		average: -586.73
Episode:   163		score: -784.66		average: -589.80
Episode:   164		score: -549.93		average: -590.82
Episode:   165		score: -418.19		average: -588.62
Episode:   166		score: -493.21		average: -588.06
Episode:   167		score: -380.39		average: -587.48
Episode:   168		score: -476.38		average: -585.47
Episode:   169		score: -530.76		average: -587.26
Episode:   170		score: -485.20		average: -585.11
Episode:   171		score: -701.02		average: -587.38
Episode:   172		score: -356.17		average: -586.36
Episode:   173		score: -441.88		average: -585.83
Episode:   174		score: -632.98		average: -587.09
Episode:   175		score: -456.85		average: -588.17
Episode:   176		score: -426.22		average: -588.07
Episode:   177		score: -861.25		average: -588.54
Episode:   178		score: -824.41		average: -592.51
Episode:   179		score: -810.63		average: -593.07
Episode:   180		score: -503.81		average: -592.96
Episode:   181		score: -435.47		average: -589.57
Episode:   182		scor

Episode:   330		score: -390.94		average: -587.11
Episode:   331		score: -454.10		average: -585.74
Episode:   332		score: -421.81		average: -585.90
Episode:   333		score: -896.42		average: -590.71
Episode:   334		score: -567.32		average: -590.26
Episode:   335		score: -385.15		average: -586.55
Episode:   336		score: -646.56		average: -587.26
Episode:   337		score: -526.81		average: -583.43
Episode:   338		score: -469.52		average: -581.63
Episode:   339		score: -476.56		average: -580.01
Episode:   340		score: -482.31		average: -580.08
Episode:   341		score: -593.16		average: -579.25
Episode:   342		score: -548.05		average: -578.03
Episode:   343		score: -722.88		average: -578.40
Episode:   344		score: -1569.67		average: -587.47
Episode:   345		score: -557.25		average: -585.78
Episode:   346		score: -412.87		average: -585.36
Episode:   347		score: -785.77		average: -584.68
Episode:   348		score: -662.75		average: -585.56
Episode:   349		score: -535.67		average: -582.33
Episode:   350		sco

Episode:   499		score: -541.60		average: -578.50
Episode:   500		score: -870.76		average: -577.58
Episode:   501		score: -735.30		average: -580.03
Episode:   502		score: -752.49		average: -581.35
Episode:   503		score: -428.84		average: -580.37
Episode:   504		score: -495.39		average: -580.89
Episode:   505		score: -510.87		average: -582.52
Episode:   506		score: -657.04		average: -583.90
Episode:   507		score: -405.34		average: -583.60
Episode:   508		score: -972.21		average: -588.69
Episode:   509		score: -474.03		average: -585.10
Episode:   510		score: -671.93		average: -583.46
Episode:   511		score: -733.65		average: -584.89
Episode:   512		score: -678.34		average: -584.78
Episode:   513		score: -410.37		average: -584.82
Episode:   514		score: -540.77		average: -585.96
Episode:   515		score: -499.19		average: -587.00
Episode:   516		score: -339.78		average: -585.85
Episode:   517		score: -519.43		average: -587.16
Episode:   518		score: -564.07		average: -588.20
Episode:   519		scor

Episode:   669		score: -820.21		average: -546.60
Episode:   670		score: -404.63		average: -542.10
Episode:   671		score: -443.59		average: -540.83
Episode:   672		score: -450.64		average: -540.52
Episode:   673		score: -435.40		average: -540.69
Episode:   674		score: -659.24		average: -542.89
Episode:   675		score: -738.69		average: -544.77
Episode:   676		score: -538.37		average: -546.89
Episode:   677		score: -539.32		average: -546.68
Episode:   678		score: -569.37		average: -546.05
Episode:   679		score: -651.80		average: -547.82
Episode:   680		score: -542.03		average: -549.37
Episode:   681		score: -586.54		average: -547.69
Episode:   682		score: -467.19		average: -549.40
Episode:   683		score: -710.50		average: -552.11
Episode:   684		score: -462.93		average: -549.39
Episode:   685		score: -759.94		average: -552.64
Episode:   686		score: -486.10		average: -550.09
Episode:   687		score: -375.49		average: -549.81
Episode:   688		score: -498.10		average: -547.89
Episode:   689		scor

...

Episode: 10092		score: -710.04		average: -562.92
Episode: 10093		score: -428.85		average: -562.63
Episode: 10094		score: -479.48		average: -563.18
Episode: 10095		score: -421.49		average: -562.70
Episode: 10096		score: -549.77		average: -562.25
Episode: 10097		score: -345.26		average: -561.94
Episode: 10098		score: -467.31		average: -562.98
Episode: 10099		score: -583.00		average: -560.39
Episode: 10100		score: -459.52		average: -560.26
Episode: 10101		score: -451.82		average: -560.19
Episode: 10102		score: -329.95		average: -558.91
Episode: 10103		score: -544.51		average: -555.63
Episode: 10104		score: -690.81		average: -555.41
Episode: 10105		score: -1036.55		average: -557.42
Episode: 10106		score: -364.77		average: -556.26
Episode: 10107		score: -561.93		average: -557.08
Episode: 10108		score: -415.18		average: -556.73
Episode: 10109		score: -778.55		average: -559.60
Episode: 10110		score: -856.06		average: -561.22
Episode: 10111		score: -722.76		average: -560.58
Episode: 10112		sco

Episode: 10261		score: -1375.90		average: -589.16
Episode: 10262		score: -600.75		average: -590.62
Episode: 10263		score: -687.48		average: -591.32
Episode: 10264		score: -704.63		average: -593.60
Episode: 10265		score: -532.67		average: -591.35
Episode: 10266		score: -793.97		average: -594.83
Episode: 10267		score: -810.30		average: -595.87
Episode: 10268		score: -619.95		average: -596.58
Episode: 10269		score: -629.01		average: -595.01
Episode: 10270		score: -483.06		average: -589.83
Episode: 10271		score: -415.16		average: -588.24
Episode: 10272		score: -812.37		average: -589.85
Episode: 10273		score: -684.82		average: -588.71
Episode: 10274		score: -434.02		average: -588.34
Episode: 10275		score: -613.36		average: -588.58
Episode: 10276		score: -675.42		average: -590.82
Episode: 10277		score: -540.50		average: -588.33
Episode: 10278		score: -521.01		average: -588.76
Episode: 10279		score: -1072.81		average: -595.57
Episode: 10280		score: -555.98		average: -597.55
Episode: 10281		sc

Episode: 10429		score: -768.94		average: -564.95
Episode: 10430		score: -510.92		average: -563.86
Episode: 10431		score: -720.42		average: -564.50
Episode: 10432		score: -915.25		average: -569.43
Episode: 10433		score: -561.99		average: -568.42
Episode: 10434		score: -790.09		average: -571.20
Episode: 10435		score: -576.76		average: -572.37
Episode: 10436		score: -672.67		average: -574.37
Episode: 10437		score: -492.09		average: -576.02
Episode: 10438		score: -371.30		average: -575.08
Episode: 10439		score: -404.66		average: -574.93
Episode: 10440		score: -547.62		average: -572.94
Episode: 10441		score: -382.74		average: -569.66
Episode: 10442		score: -479.46		average: -570.16
Episode: 10443		score: -495.67		average: -570.24
Episode: 10444		score: -425.11		average: -569.60
Episode: 10445		score: -988.33		average: -573.45
Episode: 10446		score: -517.85		average: -574.71
Episode: 10447		score: -566.40		average: -573.77
Episode: 10448		score: -605.59		average: -574.26
Episode: 10449		scor

Episode: 10597		score: -516.80		average: -564.62
Episode: 10598		score: -382.05		average: -563.81
Episode: 10599		score: -334.31		average: -562.37
Episode: 10600		score: -399.13		average: -561.77
Episode: 10601		score: -941.40		average: -567.09
Episode: 10602		score: -483.94		average: -563.89
Episode: 10603		score: -545.26		average: -559.74
Episode: 10604		score: -508.32		average: -558.00
Episode: 10605		score: -773.39		average: -561.00
Episode: 10606		score: -505.32		average: -561.89
Episode: 10607		score: -439.06		average: -562.24
Episode: 10608		score: -605.01		average: -562.88
Episode: 10609		score: -760.98		average: -560.67
Episode: 10610		score: -487.52		average: -559.61
Episode: 10611		score: -757.11		average: -562.30
Episode: 10612		score: -506.28		average: -560.68
Episode: 10613		score: -365.72		average: -557.99
Episode: 10614		score: -565.81		average: -558.85
Episode: 10615		score: -488.23		average: -559.10
Episode: 10616		score: -672.02		average: -560.92
Episode: 10617		scor

Episode: 10766		score: -492.37		average: -593.31
Episode: 10767		score: -771.92		average: -590.36
Episode: 10768		score: -560.95		average: -588.51
Episode: 10769		score: -472.73		average: -587.73
Episode: 10770		score: -498.31		average: -588.77
Episode: 10771		score: -415.38		average: -587.38
Episode: 10772		score: -583.65		average: -589.19
Episode: 10773		score: -479.05		average: -585.74
Episode: 10774		score: -497.54		average: -586.28
Episode: 10775		score: -604.26		average: -586.60
Episode: 10776		score: -572.86		average: -587.21
Episode: 10777		score: -476.15		average: -585.40
Episode: 10778		score: -574.13		average: -584.49
Episode: 10779		score: -506.34		average: -583.95
Episode: 10780		score: -489.86		average: -585.36
Episode: 10781		score: -472.91		average: -585.84
Episode: 10782		score: -694.17		average: -588.10
Episode: 10783		score: -598.86		average: -589.93
Episode: 10784		score: -437.02		average: -586.73
Episode: 10785		score: -391.82		average: -583.77
Episode: 10786		scor